# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe80ca84fa0>
├── 'a' --> tensor([[-0.2241,  0.5476,  0.0490],
│                   [ 1.0073,  1.4010, -0.9911]])
└── 'x' --> <FastTreeValue 0x7fe80ca84e50>
    └── 'c' --> tensor([[-0.3813, -0.1415, -1.7451,  0.8414],
                        [-1.5688,  1.7014, -1.3227, -1.0840],
                        [-0.7128, -0.1798, -0.4859, -0.1734]])

In [4]:
t.a

tensor([[-0.2241,  0.5476,  0.0490],
        [ 1.0073,  1.4010, -0.9911]])

In [5]:
%timeit t.a

60.3 ns ± 0.0346 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe80ca84fa0>
├── 'a' --> tensor([[ 0.4064, -2.5109,  0.3428],
│                   [ 0.9036, -0.3519, -0.2902]])
└── 'x' --> <FastTreeValue 0x7fe80ca84e50>
    └── 'c' --> tensor([[-0.3813, -0.1415, -1.7451,  0.8414],
                        [-1.5688,  1.7014, -1.3227, -1.0840],
                        [-0.7128, -0.1798, -0.4859, -0.1734]])

In [7]:
%timeit t.a = new_value

60 ns ± 0.0231 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2241,  0.5476,  0.0490],
               [ 1.0073,  1.4010, -0.9911]]),
    x: Batch(
           c: tensor([[-0.3813, -0.1415, -1.7451,  0.8414],
                      [-1.5688,  1.7014, -1.3227, -1.0840],
                      [-0.7128, -0.1798, -0.4859, -0.1734]]),
       ),
)

In [10]:
b.a

tensor([[-0.2241,  0.5476,  0.0490],
        [ 1.0073,  1.4010, -0.9911]])

In [11]:
%timeit b.a

51.8 ns ± 0.0408 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3031, -0.1033, -0.8223],
               [ 0.4740, -1.6781,  0.8902]]),
    x: Batch(
           c: tensor([[-0.3813, -0.1415, -1.7451,  0.8414],
                      [-1.5688,  1.7014, -1.3227, -1.0840],
                      [-0.7128, -0.1798, -0.4859, -0.1734]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.105 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

845 ns ± 0.139 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.7 µs ± 15.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 398 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 506 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe761739b50>
├── 'a' --> tensor([[[-0.2241,  0.5476,  0.0490],
│                    [ 1.0073,  1.4010, -0.9911]],
│           
│                   [[-0.2241,  0.5476,  0.0490],
│                    [ 1.0073,  1.4010, -0.9911]],
│           
│                   [[-0.2241,  0.5476,  0.0490],
│                    [ 1.0073,  1.4010, -0.9911]],
│           
│                   [[-0.2241,  0.5476,  0.0490],
│                    [ 1.0073,  1.4010, -0.9911]],
│           
│                   [[-0.2241,  0.5476,  0.0490],
│                    [ 1.0073,  1.4010, -0.9911]],
│           
│                   [[-0.2241,  0.5476,  0.0490],
│                    [ 1.0073,  1.4010, -0.9911]],
│           
│                   [[-0.2241,  0.5476,  0.0490],
│                    [ 1.0073,  1.4010, -0.9911]],
│           
│                   [[-0.2241,  0.5476,  0.0490],
│                    [ 1.0073,  1.4010, -0.9911]]])
└── 'x' --> <FastTreeValue 0x7fe761739c10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.7 µs ± 57.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe80ca70e80>
├── 'a' --> tensor([[-0.2241,  0.5476,  0.0490],
│                   [ 1.0073,  1.4010, -0.9911],
│                   [-0.2241,  0.5476,  0.0490],
│                   [ 1.0073,  1.4010, -0.9911],
│                   [-0.2241,  0.5476,  0.0490],
│                   [ 1.0073,  1.4010, -0.9911],
│                   [-0.2241,  0.5476,  0.0490],
│                   [ 1.0073,  1.4010, -0.9911],
│                   [-0.2241,  0.5476,  0.0490],
│                   [ 1.0073,  1.4010, -0.9911],
│                   [-0.2241,  0.5476,  0.0490],
│                   [ 1.0073,  1.4010, -0.9911],
│                   [-0.2241,  0.5476,  0.0490],
│                   [ 1.0073,  1.4010, -0.9911],
│                   [-0.2241,  0.5476,  0.0490],
│                   [ 1.0073,  1.4010, -0.9911]])
└── 'x' --> <FastTreeValue 0x7fe810af6820>
    └── 'c' --> tensor([[-0.3813, -0.1415, -1.7451,  0.8414],
                        [-1.5688,  1.7014, -1.3227, -1.0840],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 66.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.8 µs ± 79.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.3813, -0.1415, -1.7451,  0.8414],
                       [-1.5688,  1.7014, -1.3227, -1.0840],
                       [-0.7128, -0.1798, -0.4859, -0.1734]],
              
                      [[-0.3813, -0.1415, -1.7451,  0.8414],
                       [-1.5688,  1.7014, -1.3227, -1.0840],
                       [-0.7128, -0.1798, -0.4859, -0.1734]],
              
                      [[-0.3813, -0.1415, -1.7451,  0.8414],
                       [-1.5688,  1.7014, -1.3227, -1.0840],
                       [-0.7128, -0.1798, -0.4859, -0.1734]],
              
                      [[-0.3813, -0.1415, -1.7451,  0.8414],
                       [-1.5688,  1.7014, -1.3227, -1.0840],
                       [-0.7128, -0.1798, -0.4859, -0.1734]],
              
                      [[-0.3813, -0.1415, -1.7451,  0.8414],
                       [-1.5688,  1.7014, -1.3227, -1.0840],
                       [-0.7128, -0.1798, -0.4859, -0.1734]],

In [26]:
%timeit Batch.stack(batches)

133 µs ± 394 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.3813, -0.1415, -1.7451,  0.8414],
                      [-1.5688,  1.7014, -1.3227, -1.0840],
                      [-0.7128, -0.1798, -0.4859, -0.1734],
                      [-0.3813, -0.1415, -1.7451,  0.8414],
                      [-1.5688,  1.7014, -1.3227, -1.0840],
                      [-0.7128, -0.1798, -0.4859, -0.1734],
                      [-0.3813, -0.1415, -1.7451,  0.8414],
                      [-1.5688,  1.7014, -1.3227, -1.0840],
                      [-0.7128, -0.1798, -0.4859, -0.1734],
                      [-0.3813, -0.1415, -1.7451,  0.8414],
                      [-1.5688,  1.7014, -1.3227, -1.0840],
                      [-0.7128, -0.1798, -0.4859, -0.1734],
                      [-0.3813, -0.1415, -1.7451,  0.8414],
                      [-1.5688,  1.7014, -1.3227, -1.0840],
                      [-0.7128, -0.1798, -0.4859, -0.1734],
                      [-0.3813, -0.1415, -1.7451,  0.8414],
                   

In [28]:
%timeit Batch.cat(batches)

411 µs ± 7.59 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

311 µs ± 3.47 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
